### Create a database 'city' and initialise tables
    1. Create database
    2. Use the database
    3. Create tables
    4. Insert values

In [1]:
# import
import mysql.connector as msql

#Initialize Database

class CreateDatabase:
    
    def __init__(self):
        pass
    
    def connect(self):
    # Connect to the mysql database
        try:
            connection = msql.connect(
                host="localhost",
                user="root",
                password="Aakriti@4March"
            )
            # cursor = connection.cursor()
            return connection
        except msql.Error as error:
            print("Something went wrong!!{}".format())
            
            
    def createDatabase(self, cursor):
    # Creates a Database using the passed cursor
        cursor.execute("CREATE DATABASE {}".format("city"))
        
    def useDatabase(self, cursor):
        cursor.execute("USE {}".format("city"))
        
    def createTables(self, cursor):
        cursor.execute("""CREATE TABLE metadata (
        file_id INT AUTO_INCREMENT, 
        parent_id INT, 
        file_name VARCHAR(255) UNIQUE, 
        location VARCHAR(255),
        PRIMARY KEY (file_id))""")
            
        cursor.execute("""CREATE TABLE directory (
        dir_id INT AUTO_INCREMENT, 
        parent_id INT, 
        dir_name VARCHAR(255) UNIQUE, 
        PRIMARY KEY (dir_id))""")
        
    def baseDirectory(self, connection, cursor):
        cursor.execute("INSERT INTO directory(parent_id, dir_name) VALUES (0, 'base')")
        connection.commit()


if __name__ == "__main__":
    ctb = CreateDatabase()
    connection = ctb.connect()
    cursor = connection.cursor()
    cursor.execute("DROP DATABASE city")
    ctb.createDatabase(cursor)
    ctb.useDatabase(cursor)
    ctb.createTables(cursor)
    ctb.baseDirectory(connection,cursor)
    
    #cursor.execute("CREATE DATABASE {}".format("city"))
    # ctb.createDatabase(cursor)

### Connecting the notebook with mysql workbench

In [2]:
import mysql.connector

class DatabaseAccess:
    HOST = "localhost"
    USER = "root"
    PASSWORD = "Aakriti@4March"
    DATABASE = "city"

    def __init__(self) -> None:
        pass
        
    def connect(self):
        return mysql.connector.connect(
            host = self.HOST, 
            user = self.USER, 
            password = self.PASSWORD,
            database = self.DATABASE
        )
    
if __name__=="__main__":
    db = DatabaseAccess().connect()
    print(db)

In [3]:
import os 
import pandas as pd
class EDFS:
    db = DatabaseAccess().connect()
    cursor = db.cursor()
    def __init__(self) -> None:
        pass
    
    def mkdir(self, path):
        head_tail = os.path.split(path)
        tree = head_tail[0].split('/')
        parentDir = tree[-1]
        self.cursor.execute("SELECT dir_id FROM directory WHERE dir_name = '{}'".format(parentDir))
        result = self.cursor.fetchall()
        parentDirId = result[0][0]
        
        # Add the directory to the filesystem
        self.cursor.execute("INSERT INTO directory(parent_id, dir_name) VALUES({}, '{}')".format(parentDirId, head_tail[1]))
        self.db.commit()
        print("[+] Created Successfully [+]")

    def ls(self, path='base/'):
        if (path != 'base/'):
            head_tail = os.path.split(path) 
            dir = head_tail[1] 
        else:
            dir = 'base'
        self.cursor.execute("SELECT dir_id FROM directory WHERE dir_name = '{}'".format(dir))
        result = self.cursor.fetchall()
        parentId = result[0][0]
        self.cursor.execute("SELECT dir_name FROM directory WHERE parent_id = {}".format(parentId))
        result = self.cursor.fetchall()
        for i in result:
            print(i[0])
        self.cursor.execute("SELECT file_name FROM metadata WHERE parent_id = {}".format(parentId))
        result = self.cursor.fetchall()
        for i in result:
            print(i[0])
            
    def cat(self, path):
        head_tail = os.path.split(path)
        fileName = head_tail[1]
        self.cursor.execute("SELECT file_id FROM metadata WHERE file_name = '{}'".format(fileName))
        result = self.cursor.fetchall()
        fileId = result[0][0]
        self.cursor.execute("SELECT * FROM tb{}".format(fileId))
        result = self.cursor.fetchall()
        for r in result:
            print(r)
    
    def put(self, path):
        head_tail = os.path.split(path)
        fileName = head_tail[1]
        parentDir = head_tail[0].split('/')[-1]
        self.cursor.execute("SELECT dir_id FROM directory WHERE dir_name = '{}'".format(parentDir))
        result = self.cursor.fetchall()
        parentId = result[0][0]
        
        # Insert Metadata
        self.cursor.execute("INSERT INTO metadata(parent_id, file_name, location) VALUES({}, '{}', '{}')".format(parentId, fileName, head_tail[0]))
        self.db.commit()
        self.cursor.execute("SELECT file_id FROM metadata WHERE file_name = '{}'".format(fileName))
        result = self.cursor.fetchall()
        fileId = result[0][0]
        
        # Read File
        column=[]
        df = pd.read_csv(fileName,index_col=False)
        
        if(fileName == "FinalFlights.csv"):
        
            self.cursor.execute("""CREATE TABLE tb{} (
                id INT NOT NULL,
                airport_id BIGINT NOT NULL,
                city VARCHAR(100),
                state VARCHAR(100),
                name VARCHAR(100),
                dayOfMonth INT,
                dayOfWeek INT,
                carrier VARCHAR(5),
                originId BIGINT NOT NULL,
                PRIMARY KEY(id)
                )
                PARTITION BY RANGE(id) (
                    PARTITION p0 VALUES LESS THAN (5000),
                    PARTITION p1 VALUES LESS THAN (10000),
                    PARTITION P2 VALUES LESS THAN (15000),
                    PARTITION p3 VALUES LESS THAN (20000),
                    PARTITION p4 VALUES LESS THAN (30000),
                    PARTITION P5 VALUES LESS THAN MAXVALUE
                )

                """.format(fileId))
            self.db.commit()

            for i, row in df.iterrows():
                sql = "INSERT INTO tb{} VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)".format(fileId)
                self.cursor.execute(sql, tuple(row))
                self.db.commit()
        
        
        if(fileName=='Restaurant.csv'):

            self.cursor.execute("""CREATE TABLE tb{}(
                id INT NOT NULL,
                address VARCHAR(100),
                city VARCHAR(10),
                country VARCHAR(10),
                name VARCHAR(100),
                postalCode VARCHAR(100),
                province VARCHAR(10),
                PRIMARY KEY(id)
            )
            PARTITION BY RANGE(id) (
                PARTITION p0 VALUES LESS THAN (110),
                PARTITION p1 VALUES LESS THAN (120),
                PARTITION p2 VALUES LESS THAN (130),
                PARTITION p3 VALUES LESS THAN (140),
                PARTITION p4 VALUES LESS THAN MAXVALUE
            )
            """.format(fileId))
            self.db.commit()

            for i, row in df.iterrows():
                sql = "INSERT INTO tb{} VALUES(%s, %s, %s, %s, %s, %s, %s)".format(fileId)
                self.cursor.execute(sql, tuple(row))
                self.db.commit()

        print("[+] Successfully entered the file {}!!".format(fileName))
    

    def getPartition(self, file):
        self.cursor.execute("SELECT file_id FROM metadata WHERE file_name = '{}'".format(file))
        result = self.cursor.fetchall()
        id = "tb" + str(result[0][0])
        
        self.cursor.execute("""SELECT 
        PARTITION_NAME, PARTITION_DESCRIPTION 
        FROM 
        information_schema.partitions 
        WHERE TABLE_SCHEMA='city' AND TABLE_NAME = '{}' 
        AND 
        PARTITION_NAME IS NOT NULL; 
        """.format(id))
        
        result = self.cursor.fetchall()
        for i in result:
            print(i)
        

    def readPartition(self, path, partition):
        head_tail = os.path.split(path)
        fileName = head_tail[1]
        self.cursor.execute("SELECT file_id FROM metadata WHERE file_name = '{}'".format(fileName))
        result = self.cursor.fetchall()
        fileId = result[0][0]
        self.cursor.execute("SELECT * FROM tb{} PARTITION({})".format(fileId, partition))
        result = self.cursor.fetchall()
        for r in result:
            print(r)
            
    def rm(self, path):
        head_tail = os.path.split(path)
        file = head_tail[1]
        self.cursor.execute("SELECT file_id FROM metadata WHERE file_name = '{}'".format(file))
        result = self.cursor.fetchall()
        file_id = result[0][0]
        table = "tb" + str(file_id)
        
        self.cursor.execute("DELETE FROM metadata WHERE file_id ={}".format(file_id))
        self.db.commit()
        print("Deleted")

    
    

In [8]:
edfs = EDFS()
def user():
    print("Choose commands:")
    while(True):
        command = input()
        command = command.split(' ')
        if(command[0]=="mkdir"):
            edfs.mkdir(command[1])
        if(command[0]=='put'):
            edfs.put(command[1])
        if(command[0]=='ls'):
            edfs. ls(command[1])
        if(command[0]=='cat'):
            edfs.cat(command[1])
        if(command[0]=='getPartition'):
            edfs.getPartition(command[1])
        if(command[0]=='readPartition'):
            edfs.readPartition(command[1], command[2])
        else:
            break
user()

Choose commands:
put restaurants/Restaurant.csv
[+] Successfully entered the file Restaurant.csv!!


### Implementing partition-based map and reduce (PMR)

In [9]:
class MapReduce:
    db = DatabaseAccess().connect()
    cursor = db.cursor()
    def __init__(self) -> None:
        pass
    
    def mapPartition(self, p):
        
        print("What are the airports in the dataset?")
        self.cursor.execute("SELECT DISTINCT name FROM tb1  PARTITION({})".format(','.join(p)))
    
        result = self.cursor.fetchall()
        idlist = []
        for x in result:
          idlist.append(x[0])
        return idlist
        
        
    def reduce(self, p):
        result = self.mapPartition(p)
        count = len(result)
        print(count)
        
if __name__ == "__main__":
    mapreduce=MapReduce()
    result = mapreduce.mapPartition(['p0','p1','p2','p4','p5'])
    print(result)


What are the airports in the dataset?
['John F. Kennedy International', 'LaGuardia', 'Logan International', 'Seattle/Tacoma International']


In [11]:
class MapReduce:
    db = DatabaseAccess().connect()
    cursor = db.cursor()
    def __init__(self) -> None:
        pass
    
    def mapPartition(self, p):
             
        
        self.cursor.execute("SELECT COUNT(*) FROM tb1 PARTITION({}) WHERE city='New York'".format(p))
        
        result = self.cursor.fetchall()
        idlist = []
        for x in result:
          idlist.append(x[0])
        return idlist
        
        
    def reduce(self, p):
        count=0
        for i in p :
            result = self.mapPartition(i)
            print("Values in partition {}".format(i))
            print(result)
            count = count + result[0]
        print("Count of results from all partitions: ")
        print(count)
        print("The reduce function takes the values from each partition and finds the total from each partition")
        
if __name__ == "__main__":
    print("Calculate the number of flights that landed in New York?")
    mapreduce=MapReduce()
    result = mapreduce.reduce(['p0','p1','p2','p3','p4'])
    


Calculate the number of flights that landed in New York?
Values in partition p0
[4900]
Values in partition p1
[5000]
Values in partition p2
[5000]
Values in partition p3
[5000]
Values in partition p4
[100]
Count of results from all partitions: 
20000
The reduce function takes the values from each partition and finds the total from each partition


In [12]:
class MapReduce:
    db = DatabaseAccess().connect()
    cursor = db.cursor()
    def __init__(self) -> None:
        pass
    
    def mapPartition(self, p):
        print(" Calculate the number of Burger King in New York?")
        self.cursor.execute("SELECT COUNT(*) FROM tb2 PARTITION({}) WHERE city='New York' AND name='Burger King'"
                            .format(','.join(p)))
        result = self.cursor.fetchall()
        idlist = []
        for x in result:
          idlist.append(x[0])
        return idlist
        
        
    def reduce(self, p):
        result = self.mapPartition(p)
        count = len(result)
        print(count)
        
if __name__ == "__main__":
    mapreduce=MapReduce()
    result = mapreduce.mapPartition(['p0','p1','p2','p4'])
    print(result)

 Calculate the number of Burger King in New York?
[5]


In [15]:
class MapReduce:
    db = DatabaseAccess().connect()
    cursor = db.cursor()
    def __init__(self) -> None:
        pass
    
    def mapPartition(self, p):
        print("Calculate the number of AMERICAN AIRLINES landing in Boston on a Tuesday?")
        self.cursor.execute("SELECT COUNT(*) FROM tb1 PARTITION({}) WHERE city='Boston' AND DayOfWeek=3 AND Carrier='AA'".format(','.join(p)))
        
        result = self.cursor.fetchall()
        idlist = []
        for x in result:
          idlist.append(x[0])
        return idlist
    def reduce(self, p):
        result = self.mapPartition(p)
        count = len(result)
        print(count)
        
if __name__ == "__main__":
    mapreduce=MapReduce()
    result = mapreduce.mapPartition(['p0','p1','p2','p4','p5'])
    print(result)



Calculate the number of AMERICAN AIRLINES landing in Boston on a Tuesday?
[132]


In [14]:
class MapReduce:
    db = DatabaseAccess().connect()
    cursor = db.cursor()
    def __init__(self) -> None:
        pass
    
    def mapPartition(self, p):
        print("What are the restaurants in Seattle?")
        self.cursor.execute("SELECT DISTINCT name FROM tb2 PARTITION({}) WHERE city='Seattle'".format(','.join(p)))

    
        result = self.cursor.fetchall()
        idlist = []
        for x in result:
          idlist.append(x[0])
        return idlist
    
    def reduce(self, p):
        result = self.mapPartition(p)
        count = len(result)
        print(count)
        
if __name__ == "__main__":
    mapreduce=MapReduce()
    result = mapreduce.mapPartition(['p0','p1','p2','p4'])
    print(result)
    print("")
    
import ipywidgets as widgets
widgets.Dropdown(
    options=result,
    value=result[0],
    description='Restaurants:',
    disabled=False,
)

What are the restaurants in Seattle?
["McDonald's", 'Baskin-Robbins', 'Five Guys', 'Burger King', 'Taco Del Mar', 'SUBWAY', 'Pizza Hut', "Dino's Gyros", 'KFC']



Dropdown(description='Restaurants:', options=("McDonald's", 'Baskin-Robbins', 'Five Guys', 'Burger King', 'Tac…